<br>
サイドメニューのFile BrowserからTwitterのアーカイブ（ZIPファイル）をtmpディレクトリに「アップロード」（実際にはローカルコピーであり、外部ネットワークへの送信ではありません）した後、上にある早送りボタン（「Restart Kernel and Run All Cells...」）を押してすべてのセルを実行してください。しばらく待つと、この直下のセルのさらに下に入力欄が表示されますので、アップロードしたファイル名（パスではなくファイル名のみ。ディレクトリ名は不要）を入力してください。

In [ ]:
import pathlib
import shutil

tmp = pathlib.Path('tmp')

for _ in range(5):
    file_name = input()
    file_path = tmp.joinpath(file_name)

    if file_path.is_file():
        break

    print("入力されたファイル名のファイルが存在しません。アーカイブ（ZIPファイル）をtmpディレクトリにアップロードし、正しいファイル名を入力してください。")

    # コピペミスの可能性の通知
    if '.' not in file_name:
        print()
        print("もしかしたら拡張子が抜けているせいかもしれません。確認してみてください。")
else:
    print()
    print("入力されたファイル名のファイルが存在しませんでした。アーカイブ（ZIPファイル）をtmpディレクトリにアップロードし、ファイル名を確認した上でこのセルを再実行し、正しいファイル名を入力してください。")

unzipped_directory_name = file_path.stem
unzipped_directory_path = tmp.joinpath(unzipped_directory_name)
shutil.unpack_archive(file_path, unzipped_directory_path)

この上に表示される入力欄にアップロードしたファイル名を入力してください。ファイル名が正しく入力されていれば分析結果がページ最下部に表示されます。

In [ ]:
import pathlib
import json

read_path = pathlib.Path().joinpath(unzipped_directory_path, 'data', 'tweets.js')
write_path = tmp.joinpath('tweets.json')

content = read_path.read_text(encoding='utf-8')
content = content.replace('window.YTD.tweets.part0 = ', '', 1)
write_path.write_text(content, encoding='utf-8')

with open(write_path, encoding='utf-8') as f:
    df = json.load(f)

print(len(df))